In [291]:
import scipy.io as io
import numpy as np

In [292]:
dataset = io.loadmat('Indian_pines.mat')['indian_pines']
gt_labels = io.loadmat('Indian_pines_gt.mat')['indian_pines_gt']
print(dataset[0][0])
print(gt_labels[0][0])

[3172 4142 4506 4279 4782 5048 5213 5106 5053 4750 4816 4769 4610 4805
 4828 4861 4767 4624 4549 4463 4462 4446 4445 4336 4381 4319 4207 4305
 4311 3991 4168 3942 4061 4362 4318 4252 4869 5284 5055 3591 5175 5217
 5058 4969 4721 4291 4555 4886 4868 4806 4783 4811 4709 3903 3795 3715
 2591 2130 2269 2480 3145 3626 4060 4296 4211 4225 4157 4133 4082 4048
 3935 3843 3784 3642 3271 2707 1707 1564 1838 1719 2229 2764 2919 2873
 2977 2913 3034 3051 3124 3101 3033 2713 2740 2947 2706 2834 2856 2683
 2400 2229 1822 1542 1097 1029 1020 1026 1009 1011 1047 1069 1100 1122
 1259 1365 1261 1374 1630 1851 2028 2130 2170 2205 2214 2204 2100 2106
 2146 2089 2078 2134 2127 2074 2057 2045 2003 1999 1959 1924 1883 1843
 1781 1716 1698 1645 1540 1410 1294 1131 1044 1029 1006 1017 1000  995
  997 1003 1016 1001 1003 1002 1005 1004 1008 1032 1045 1100 1212 1295
 1244 1100 1103 1216 1346 1330 1259 1251 1313 1372 1393 1402 1396 1381
 1396 1381 1353 1346 1341 1332 1324 1310 1318 1330 1310 1292 1280 1275
 1266 

In [293]:
a = {x:[] for x in range(1,17)}
(x,y,z) = dataset.shape
for i in range(x):
    for j in range(y):
        l = gt_labels[i][j]
        if l!=0:
            a[l].append(np.array(dataset[i][j]))

In [294]:
for i in range(1,17):
    print(len(a[i]))

46
1428
830
237
483
730
28
478
20
972
2455
593
205
1265
386
93


In [295]:
y = []
X = []
count=0
for i in range(1,17):
    if(len(a[i])>480):
        count+=1
        for j in range(len(a[i])):
            X.append(a[i][j])
            y.append(count)
X = np.array(X)
y = np.array(y)


In [296]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.utils import to_categorical
X = normalize(X,axis=1,return_norm=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
X_test = X_test.reshape(len(X_test),220,1,1)
X_train = X_train.reshape(len(X_train),220,1,1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train = y_train[:,1:]
y_test = y_test[:,1:]


In [297]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Flatten, MaxPooling2D , Dropout
from keras.initializers import RandomUniform
from keras.callbacks import ModelCheckpoint

In [298]:
from keras.initializers import RandomUniform
def model_generator(n1,k1,k2,n4,n5,n_k,a1,a2,a3):
    initializer = RandomUniform(minval=-0.05, maxval=0.05)
    model = Sequential()
    model.add(Conv2D(n_k, kernel_size=(k1,1), activation=a1, input_shape=(n1,1,1),kernel_initializer=initializer,bias_initializer='zeros'))
    model.add(MaxPooling2D(pool_size = (k2,1)))
    model.add(Flatten())
    model.add(Dense(n4,activation=a2,kernel_initializer=initializer))
    model.add(Dense(n5,activation = a3))
    return model

In [299]:
model=model_generator(220,24,5,100,count,20,'tanh','tanh','softmax')
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [300]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 75,verbose=1,batch_size=8,callbacks=callbacks_list)

Train on 7880 samples, validate on 876 samples
Epoch 1/75
7880/7880 [==============================] - 5s 692us/step - loss: 1.4348 - acc: 0.4487 - val_loss: 1.2962 - val_acc: 0.4795

Epoch 00001: val_acc improved from -inf to 0.47945, saving model to weights-improvement-01-0.48.hdf5
Epoch 2/75
7880/7880 [==============================] - 4s 529us/step - loss: 1.2648 - acc: 0.4963 - val_loss: 1.2368 - val_acc: 0.5114

Epoch 00002: val_acc improved from 0.47945 to 0.51142, saving model to weights-improvement-02-0.51.hdf5
Epoch 3/75
7880/7880 [==============================] - 4s 525us/step - loss: 1.2106 - acc: 0.5178 - val_loss: 1.2520 - val_acc: 0.5103

Epoch 00003: val_acc did not improve from 0.51142
Epoch 4/75
7880/7880 [==============================] - 4s 483us/step - loss: 1.1434 - acc: 0.5487 - val_loss: 1.1542 - val_acc: 0.5445

Epoch 00004: val_acc improved from 0.51142 to 0.54452, saving model to weights-improvement-04-0.54.hdf5
Epoch 5/75
7880/7880 [========================


Epoch 00038: val_acc did not improve from 0.85731
Epoch 39/75
7880/7880 [==============================] - 4s 494us/step - loss: 0.4005 - acc: 0.8496 - val_loss: 0.3643 - val_acc: 0.8630

Epoch 00039: val_acc improved from 0.85731 to 0.86301, saving model to weights-improvement-39-0.86.hdf5
Epoch 40/75
7880/7880 [==============================] - 4s 483us/step - loss: 0.3898 - acc: 0.8577 - val_loss: 0.3543 - val_acc: 0.8801

Epoch 00040: val_acc improved from 0.86301 to 0.88014, saving model to weights-improvement-40-0.88.hdf5
Epoch 41/75
7880/7880 [==============================] - 4s 473us/step - loss: 0.3939 - acc: 0.8504 - val_loss: 0.4099 - val_acc: 0.8516

Epoch 00041: val_acc did not improve from 0.88014
Epoch 42/75
7880/7880 [==============================] - 4s 470us/step - loss: 0.3869 - acc: 0.8549 - val_loss: 0.3453 - val_acc: 0.8881

Epoch 00042: val_acc improved from 0.88014 to 0.88813, saving model to weights-improvement-42-0.89.hdf5
Epoch 43/75
7880/7880 [============